In [44]:
#https://qiskit.org/textbook/ch-applications/vqe-molecules.html#Running-VQE-on-the-statevector-Simulator

# pylint: disable=line-too-long
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import (GroundStateEigensolver,
                                      NumPyMinimumEigensolverFactory)
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureMoleculeDriver, ElectronicStructureDriverType)
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer, ActiveSpaceTransformer
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper
# pylint: enable=line-too-long

import matplotlib.pyplot as plt
import numpy as np
from qiskit_nature.circuit.library import UCCSD, HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP
from qiskit.opflow import TwoQubitReduction
from qiskit import BasicAer, Aer
from qiskit.utils import QuantumInstance
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

In [77]:
def get_qubit_op(dist):
    # Define Molecule
    molecule = Molecule(
        # Coordinates in Angstrom
        geometry=[
["Co",[6.844708500129254,      5.772690500054138,      5.000000000000000]],
["O",[8.228239500226193,      6.352208500094744,      5.000000000000000]],
["C",[9.292723500300781,      6.822059500127666,      5.000000000000000]],
["O",[10.323201500372985,      7.276646500159519,      5.000000000000000]]
        ],
        multiplicity=1,  # = 2*spin + 1
        charge=1,
    )
    
    driver = ElectronicStructureMoleculeDriver(
        molecule=molecule,
        basis="sto3g",
        driver_type=ElectronicStructureDriverType.PYSCF)

    # Get properties
    properties = driver.run()
    print("Debug data: ")
    print(dir(properties), properties._properties, properties.molecule)
    
    num_particles = (properties
                        .get_property("ParticleNumber")
                        .num_particles)
    num_spin_orbitals = int(properties
                            .get_property("ParticleNumber")
                            .num_spin_orbitals)
    print(num_particles)
    print(num_spin_orbitals)

    print("Freezing CORE Orbitals")
    fc_transformer = FreezeCoreTransformer()

    fc_problem = fc_transformer.transform(properties)
    num_particles = (fc_problem
                        .get_property("ParticleNumber")
                        .num_particles)
    num_spin_orbitals = int(fc_problem
                            .get_property("ParticleNumber")
                            .num_spin_orbitals)
    print(num_particles)
    print(num_spin_orbitals)
    print(dir(properties), properties._properties, properties.molecule)
    #    print(fc_problem.hamiltonian.constants)

    print("Freezing Custom Orbitals")
    fc_transformer = FreezeCoreTransformer(remove_orbitals=range(13,33))

    fc_problem = fc_transformer.transform(properties)
    num_particles = (fc_problem
                        .get_property("ParticleNumber")
                        .num_particles)
    num_spin_orbitals = int(fc_problem
                            .get_property("ParticleNumber")
                            .num_spin_orbitals)
    print(num_particles)
    print(num_spin_orbitals)
    #print(fc_problem.hamiltonian.constants)

    print("Active space transformer")
    as_transformer = ActiveSpaceTransformer(6, 6)
    as_problem = as_transformer.transform(properties)
    num_particles = (fc_problem
                        .get_property("ParticleNumber")
                        .num_particles)
    num_spin_orbitals = int(fc_problem
                            .get_property("ParticleNumber")
                            .num_spin_orbitals)
    print(num_particles)
    print(num_spin_orbitals)
    #print(as_problem.hamiltonian.electronic_integrals.alpha)

    print("Active space transformer SET Active orbitals")
    as_transformer = ActiveSpaceTransformer(6, 6)#, active_orbitals=[30, 33])
    as_problem = as_transformer.transform(properties)
    num_particles = (fc_problem
                        .get_property("ParticleNumber")
                        .num_particles)
    num_spin_orbitals = int(fc_problem
                            .get_property("ParticleNumber")
                            .num_spin_orbitals)
    print(num_particles)
    print(num_spin_orbitals)
    #print(as_problem.hamiltonian.electronic_integrals.alpha)

    #stop;
    
    # Define Problem, Use freeze core approximation, remove orbitals.
    problem = ElectronicStructureProblem(
        driver,
        [FreezeCoreTransformer(freeze_core=True,
                               #remove_orbitals=range(13,33)
                              ),
         #ActiveSpaceTransformer(6, 6)#, active_orbitals=[30,33])
        ])

    second_q_ops = problem.second_q_ops()  # Get 2nd Quant OP
    num_spin_orbitals = problem.num_spin_orbitals
    num_particles = problem.num_particles
         

    mapper = ParityMapper()  # Set Mapper
    print(second_q_ops)
    hamiltonian = second_q_ops['ElectronicEnergy']  # Set Hamiltonian
    
    print(dir(hamiltonian))
    print(hamiltonian.terms)
    #print(problem.hamiltonian.electronic_integrals.alpha)

    # Do two qubit reduction
    converter = QubitConverter(mapper,two_qubit_reduction=True)
    reducer = TwoQubitReduction(num_particles)
    qubit_op = converter.convert(hamiltonian)
    print("Before reduction ", hamiltonian, qubit_op)
    qubit_op = reducer.convert(qubit_op)
    print("After reduction ", qubit_op)

    return qubit_op, num_particles, num_spin_orbitals, problem, converter


In [ ]:
def exact_solver(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

backend = BasicAer.get_backend("statevector_simulator")
distances = np.arange(0.5, 3.0, 1)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=5)

# pylint: disable=undefined-loop-variable
for dist in distances:
    (qubit_op, num_particles, num_spin_orbitals,
                             problem, converter) = get_qubit_op(dist)
    result = exact_solver(problem,converter)
    exact_energies.append(result.total_energies[0].real)
    print(exact_energies)

    '''
    init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
    var_form = UCCSD(converter,
                     num_particles,
                     num_spin_orbitals,
                     initial_state=init_state)
    vqe = VQE(var_form, optimizer, quantum_instance=backend)
    vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
    vqe_result = problem.interpret(vqe_calc).total_energies[0].real
    vqe_energies.append(vqe_result)
    print(f"Interatomic Distance: {np.round(dist, 2)}",
          f"VQE Result: {vqe_result:.5f}",
          f"Exact Energy: {exact_energies[-1]:.5f}")
    '''
print("All energies have been calculated")


Debug data: 
['VERSION', '__abstractmethods__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_generator', '_is_protocol', '_name', '_properties', '_validate_input_type', 'add_property', 'from_hdf5', 'from_legacy_driver_result', 'get_property', 'interpret', 'log', 'molecule', 'name', 'second_q_ops', 'to_hdf5'] {'DriverMetadata': <qiskit_nature.properties.second_quantization.driver_metadata.DriverMetadata object at 0x7f3c2d7d7760>, 'ElectronicBasisTransform': <qiskit_nature.properties.second_quantization.electronic.bases.electronic_basis_transform.ElectronicBasisTransform object at 0x7f3c3afee190>, '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



After reduction  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

<ipython-input-78-4dbf3aa4787d>:3: NatureDeprecationWarning: The qiskit_nature.algorithms.ground_state_solvers.GroundStateEigensolver class is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.ground_state_solvers.GroundStateEigensolver class.
  calc = GroundStateEigensolver(converter, solver)


In [ ]:
plt.plot(distances, exact_energies, label="Exact Energy")
plt.plot(distances, vqe_energies, label="VQE Energy")
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy')
plt.legend()
plt.show()


In [ ]:
print(exact_energies, init_state)
